In [1]:
#Ya Khode Khoda

In [2]:
import pandas as pd 
import numpy as np
from IPython.display import clear_output
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import matplotlib.cm as cm
import matplotlib as matplotlib

from fastdtw import fastdtw

def color_map_color(value, cmap_name='coolwarm', vmin=0, vmax=10):
    # norm = plt.Normalize(vmin, vmax)
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)  # PiYG
    rgb = cmap(norm(abs(value)))[:3]  # will return rgba, we take only first 3 so we get rgb
    color = matplotlib.colors.rgb2hex(rgb)
    return color
cl=['r','g','b','c','m','y','k']
color=dict()
for i,el in enumerate(cl):
    color.update({i:el})
#extract the labels for clutering precision its for after clustering
def time_convertor(x):
    s,h,m=0,0,0
    s=round(x%60,2)
    m=int(x/60)
    h=int(m/60)
    m=m%60
    time=str(h)+':'+str(m)+':'+str(s)
    return pd.to_datetime('1970-01-01 '+time)


# Display figures inline in Jupyter notebook
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 5)})

In [3]:
#data_read
#prepare Dataset gyroscope
dfg=pd.read_csv('16\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('16\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
dfg['g-x']=dfg.apply(lambda x:x[0]/max(dfg['g-x']),axis=1)
dfg['g-y']=dfg.apply(lambda x:x[1]/max(dfg['g-y']),axis=1)
dfg['g-z']=dfg.apply(lambda x:x[2]/max(dfg['g-z']),axis=1)
dfa['a-x']=dfg.apply(lambda x:x[0]/max(dfa['a-x']),axis=1)
dfa['a-y']=dfg.apply(lambda x:x[1]/max(dfa['a-y']),axis=1)
dfa['a-z']=dfg.apply(lambda x:x[2]/max(dfa['a-z']),axis=1)
df1 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('17\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('17\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
dfg['g-x']=dfg.apply(lambda x:x[0]/max(dfg['g-x']),axis=1)
dfg['g-y']=dfg.apply(lambda x:x[1]/max(dfg['g-y']),axis=1)
dfg['g-z']=dfg.apply(lambda x:x[2]/max(dfg['g-z']),axis=1)
dfa['a-x']=dfg.apply(lambda x:x[0]/max(dfa['a-x']),axis=1)
dfa['a-y']=dfg.apply(lambda x:x[1]/max(dfa['a-y']),axis=1)
dfa['a-z']=dfg.apply(lambda x:x[2]/max(dfa['a-z']),axis=1)
df2 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('20\\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('20\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
dfg['g-x']=dfg.apply(lambda x:x[0]/max(dfg['g-x']),axis=1)
dfg['g-y']=dfg.apply(lambda x:x[1]/max(dfg['g-y']),axis=1)
dfg['g-z']=dfg.apply(lambda x:x[2]/max(dfg['g-z']),axis=1)
dfa['a-x']=dfg.apply(lambda x:x[0]/max(dfa['a-x']),axis=1)
dfa['a-y']=dfg.apply(lambda x:x[1]/max(dfa['a-y']),axis=1)
dfa['a-z']=dfg.apply(lambda x:x[2]/max(dfa['a-z']),axis=1)
df3 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('21\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('21\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
dfg['g-x']=dfg.apply(lambda x:x[0]/max(dfg['g-x']),axis=1)
dfg['g-y']=dfg.apply(lambda x:x[1]/max(dfg['g-y']),axis=1)
dfg['g-z']=dfg.apply(lambda x:x[2]/max(dfg['g-z']),axis=1)
dfa['a-x']=dfg.apply(lambda x:x[0]/max(dfa['a-x']),axis=1)
dfa['a-y']=dfg.apply(lambda x:x[1]/max(dfa['a-y']),axis=1)
dfa['a-z']=dfg.apply(lambda x:x[2]/max(dfa['a-z']),axis=1)
df4 = pd.concat([dfa, dfg], axis=1, join='outer')

In [4]:
label_event_lenght=dict()
#prepare Labeled dataset on 16
dfl=pd.read_csv('16\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})
        
dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_1=list()
for i in range(len(dfl)):
    evented_label_1.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))
#prepare Labeled dataset on 17
dfl=pd.read_csv('17\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})

dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_2=list()
for i in range(len(dfl)):
    evented_label_2.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))
#prepare Labeled dataset on 20
dfl=pd.read_csv('20\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})

dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_3=list()
for i in range(len(dfl)):
    evented_label_3.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))
#prepare Labeled dataset on 21
dfl=pd.read_csv('21\\groundTruth.csv')
dfl['length']=dfl.en-dfl.st
for event , dft in dfl.groupby('evento'):
    if (event in label_event_lenght):
        label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
    else:
        label_event_lenght.update({event : list(dft.length)})
dfl['st_time']=dfl.apply(lambda x:time_convertor(x.st) ,axis=1 )
dfl['en_time']=dfl.apply(lambda x:time_convertor(x.en) ,axis=1 )
dfl=dfl.drop('st',1).drop('en',1).drop('length',1)
evented_label_4=list()
for i in range(len(dfl)):
    evented_label_4.append((dfl.iloc[i][0],dfl.iloc[i][1],dfl.iloc[i][2]))

eve1=pd.DataFrame(evented_label_1)
eve1[3]=['evented_label_1' for x in evented_label_1]

eve2=pd.DataFrame(evented_label_2)
eve2[3]=['evented_label_2' for x in evented_label_2]

eve3=pd.DataFrame(evented_label_3)
eve3[3]=['evented_label_3' for x in evented_label_3]

eve4=pd.DataFrame(evented_label_4)
eve4[3]=['evented_label_4' for x in evented_label_4]

evented_label_All=pd.concat([eve1, eve2,eve3,eve4])

#supervised
window={226:['curva_direita_agressiva','curva_esquerda_agressiva','aceleracao_agressiva'],
        126:['troca_faixa_direita_agressiva','troca_faixa_esquerda_agressiva'],
        186:['freada_agressiva']
       }

In [5]:
#optimaze the forward step for the labeled dataset
#param for optimization


t1=time.time()
def TempEvent_genarator_new(Lambda,df,L,i):
    M=int((L)*Lambda)
    temp_event=list()
    pointer=0
    stop=False
    #go on timeserise for event extraction
    while stop!=True:
        clear_output(wait=True)
        pointer=pointer + M
        l_min=pointer-int(L/2)
        l_max=pointer+int(L/2)
        if l_max<0:
            l_max=L/2
        if (l_min<0):
            l_min=0
        if (l_max>len(df)):
            l_max=len(df)
            stop=True
        print('extraction ',l_max/len(df),' % of ',i,' is complete')
        temp_event.append(df[l_min:l_max])
    return temp_event

def has_overlap(A_start, A_end, B_start, B_end):
    latest_start = max(A_start, B_start)
    earliest_end = min(A_end, B_end)
    return latest_start <= earliest_end

def export_labels(tempevent,evented_label):
    EvL=pd.DataFrame(evented_label)
    Labels={
          226:[],
          126:[],
          186:[],
           }
    for i,wind in enumerate(Labels):
        label=['NAG' for x in tempevent[i][1]]
        for name in tempevent[i][0]:
            el=[(x[1],x[2]) for x in EvL.iloc if x[0]==name]
            for j,te in enumerate(tempevent[i][1]):
                    sta =te.index[0]
                    ena =te.index[-1]
                    for stb,enb in el:
                        if has_overlap(sta,ena,stb,enb):
                            label[j]=name
        Labels[wind]=label
    return Labels




LAMBDA=list()

for Lambda in in np.linspace(0.1,1,10):
    temp_event_test=list()
    for l,DF in enumerate([df1,df2,df3,df4]):
        te=list()
        for i,lw in enumerate(window):
            te.append((window[lw],TempEvent_genarator_new(Lambda,DF,lw,i+1)))
        temp_event_test.append(te)

    #make it dataframe
    Ev=[evented_label_1,evented_label_2,evented_label_3,evented_label_4]
    Data_test={
          226:[],
          126:[],
          186:[],
           }
    for i,temp in enumerate(temp_event_test):
        TempEv={
                  226:[ x for x in temp_event_test[i][0][1] ],
                  126:[ x for x in temp_event_test[i][1][1] ],
                  186:[ x for x in temp_event_test[i][2][1] ],
                   }
        Label=export_labels(temp_event_test[i],Ev[i])
        #merge together
        for wind in Data_test:
            for j in range(len(TempEv[wind])):
                Data_test[wind].append((Label[wind][j],TempEv[wind][j]))

    #calculate distance
    k,n=0,0
    num_event=dict(evented_label_All.groupby(0).size())
    for lenght in window:
        for name in window[lenght]:
            k=k+len(Data_test[lenght])*num_event[name]

    Data_DS_test={
          226:[],
          126:[],
          186:[],
           }

    for wind in Data_test:
        for label,event in Data_test[wind]:
             #---------for each Data-distance from events----------
            Y=list()
            for from_event,dfe in evented_label_All.groupby(0):
                if from_event in window[wind]:
                    X=list()
                    #____for each event_distance from each label_____
                    for el in dfe.iloc:
                        clear_output(wait=True)
                        n=n+1
                        print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
                        if   el[3]=='evented_label_1':
                            X.append(fastdtw(event,df1[el[1]:el[2]])[0])
                        elif el[3]=='evented_label_2':
                            X.append(fastdtw(event,df2[el[1]:el[2]])[0])
                        elif el[3]=='evented_label_3':
                            X.append(fastdtw(event,df3[el[1]:el[2]])[0])
                        elif el[3]=='evented_label_4':
                            X.append(fastdtw(event,df4[el[1]:el[2]])[0])
                    #_________________________________________________
                    Y.append((from_event,X))
            Data_DS_test[wind].append((label,Y))
            #------------------------------------------------------

    #prepare dataset for labeling
    TrainData_test=dict()
    for win in Data_DS_test:
        x=Data_DS_test[win]
        dumy_list=list()
        for events in x:
            dumy_dict=dict()
            dumy_dict.update({'label':events[0]})
            for name , event in events[1]:
                for tag,number in enumerate(event):
                    dumy_dict.update({name+str(tag):number})
            dumy_list.append(dumy_dict)
        TrainData_test.update({win:pd.DataFrame(dumy_list)})
    LAMBDA.append((Lambda,TrainData_test))

with open("TrainData_test_Optimize_Forward_step.txt", "wb") as fp:
    pickle.dump(TrainData_test, fp)
t2=time.time()
print(round((t2-t1)/60) ,'min has time left')

disstance calculatoin  100.0  %  is complete
52 min has time left


array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [17]:
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report

with open("OverFit_model.txt", "rb") as fp:
    Result = pickle.load(fp)
    
#with open("TrainData_test_l_1.txt", "rb") as fp:
    #temp_event_test,TrainData_test = pickle.load(fp)

In [18]:
wid=226
Max_depth=8

data=TrainData_test[wid].drop('label',1)
label=TrainData_test[wid].label

estimator=Result[wid][1]
print(classification_report(label, estimator.predict(data)))


                          precision    recall  f1-score   support

                     NAG       0.96      0.96      0.96       630
    aceleracao_agressiva       0.47      0.35      0.40        23
 curva_direita_agressiva       0.63      0.60      0.62        20
curva_esquerda_agressiva       0.64      0.74      0.68        19

                accuracy                           0.93       692
               macro avg       0.67      0.66      0.66       692
            weighted avg       0.92      0.93      0.92       692

